In [ ]:
BASEDIR = '.'
TRAINDATAPATH = 'dataset_covid_pneumonia/train'
TESTDATAPATH = 'dataset_covid_pneumonia/test'
VALIDDATAPATH = 'dataset_covid_pneumonia/valid'

In [ ]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,\
                                    MaxPooling2D,BatchNormalization,\
                                    MaxPooling2D,AveragePooling2D,\
                                    GlobalMaxPooling2D,GlobalAveragePooling2D,\
                                    Concatenate,Input,ZeroPadding2D,Reshape
                                    
from keras.layers.merge import concatenate
import numpy as np
import time
import os

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.callbacks import TensorBoard

from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model

from tensorflow.keras.applications.resnet import preprocess_input

from tensorflow.keras.optimizers import Adam 


import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.densenet import DenseNet121,DenseNet169,DenseNet201
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model

In [ ]:
#train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)#,rotation_range=30,shear_range=0.25,zoom_range=0.1, horizontal_flip = True) 
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   shear_range=0.25,
                                   zoom_range=0.1, 
                                   horizontal_flip = True
                                   )
train_generator=train_datagen.flow_from_directory(TRAINDATAPATH, 
                                                 target_size=(224,224),
                                                 batch_size=1,
                                                 class_mode="categorical",
                                                 shuffle=True)


valid_generator=train_datagen.flow_from_directory(VALIDDATAPATH, 
                                                 target_size=(224,224),
                                                 batch_size=1,
                                                 class_mode="categorical",
                                                 shuffle=True)

eval_generator = train_datagen.flow_from_directory(TESTDATAPATH,
                                                  target_size=(224,224),
                                                  batch_size=1,
                                                  shuffle=True,
                                                  seed=42,
                                                  class_mode="categorical")

In [ ]:
print(train_generator.class_indices)

In [ ]:
import matplotlib.pyplot as plt


def plot_acc_loss(result, epochs):
    acc = result.history['accuracy']
    loss = result.history['loss']
    val_acc = result.history['val_accuracy']
    val_loss = result.history['val_loss']
    plt.figure(figsize=(15, 5))
    plt.subplot(121)
    plt.plot(range(1,epochs), acc[1:], label='Train_acc')
    plt.plot(range(1,epochs), val_acc[1:], label='Test_acc')
    plt.title('Accuracy over ' + str(epochs) + ' Epochs', size=15)
    plt.legend()
    plt.grid(True)
    plt.subplot(122)
    plt.plot(range(1,epochs), loss[1:], label='Train_loss')
    plt.plot(range(1,epochs), val_loss[1:], label='Test_loss')
    plt.title('Loss over ' + str(epochs) + ' Epochs', size=15)
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
from tensorflow.keras.applications.resnet import preprocess_input
import os
from tensorflow.keras import optimizers

denseNumber = 3
denseNodes = 64
dropoutNumber = 0.25
batchSize = 2
epochNumber = 100

LOGNAME = "DenseNet121_RUN_REAL_2.0_DenseLayers-{0}_DenseNodes-{1}_Dropout-{2}_BatchSize-{3}_EpochNumber-{4}".format(denseNumber,denseNodes,dropoutNumber,batchSize,epochNumber)
log_dir = os.path.join(BASEDIR,"logs",LOGNAME,"fit",)
tensorboard = TensorBoard(log_dir = log_dir)
print(LOGNAME)


#base_model=ResNet50(include_top=False, weights='imagenet')
base_model=DenseNet121(include_top=False, weights='imagenet',pooling="avg")
base_model.trainable = False

x=base_model.output
#x=GlobalAveragePooling2D()(x)
x=Flatten()(x)
for i in range(denseNumber-1):
  x=Dense(denseNodes,activation='relu')(x) 
  x=Dropout(dropoutNumber)(x) 
  

preds=Dense(3,activation='sigmoid')(x)

model=Model(inputs=base_model.input,outputs=preds)

model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(lr=0.0005),metrics=['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.TruePositives(),  tf.keras.metrics.TrueNegatives(),  tf.keras.metrics.FalsePositives(),  tf.keras.metrics.FalseNegatives()])

step_size_train=train_generator.n//train_generator.batch_size
step_size_valid=valid_generator.n//valid_generator.batch_size
history=model.fit_generator(train_generator,
                    steps_per_epoch =step_size_train,
                    validation_data = valid_generator,
                    validation_steps = step_size_valid,
                    epochs= epochNumber,
                    callbacks=[tensorboard]
                    )
plot_acc_loss(history, epochNumber)
print('TEST')

eval_generator.reset() 
x = model.evaluate_generator(eval_generator,
                  steps = np.ceil(len(eval_generator) / 1),
                  use_multiprocessing = False,
                  verbose = 1,
                  workers=1
                  )

print('Test loss:' , x[0])
print('Test accuracy:',x[1])
print('Test precision:', x[2])
print('Test Recall:',x[3])
print('Test F1 Score: ',2*(x[2]*x[3])/(x[2]+x[3]))

In [ ]:
len(eval_generator)

In [ ]:
eval_generator.reset()  
pred = model.predict_generator(eval_generator,steps = len(eval_generator),verbose=1)
print("Predictions finished")

In [ ]:
import cv2

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
tmp_lst=[]
for index, probability in enumerate(pred):
    print(probability)
    arg=np.argmax(probability)
    
    image_path = TESTDATAPATH + "/" +eval_generator.filenames[index]
    image = mpimg.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    pixels = np.array(image)
    plt.imshow(pixels)
    
    print()
    if arg == 0:
        tmp_lst.append((eval_generator.filenames[index],'covid-19'))
        plt.title("%.2f" % (probability[arg]*100) + "% COVID19")
    elif arg == 1:
        tmp_lst.append((eval_generator.filenames[index],'healthy'))
        plt.title("%.2f" % (probability[arg]*100) + "% Healthy")
    else:
        tmp_lst.append((eval_generator.filenames[index],'pneumonia'))
        plt.title("%.2f" % (probability[arg]*100) + "% Pneumonia")
    plt.show()

In [ ]:
tmp_lst

In [ ]:
import numpy as np
arg=np.argmax(np.array([7.7917069e-01,2.5371173e-06, 8.7350083e-04]))

In [ ]:
arg